In [1]:
import openai
import os
import pandas as pd
import time
text_file = open("API_key.txt", "r")

folder_path = 'repo' # Replace with the path to your folder
file_paths_details = []

select_files = input('Update File List? : ')

def highlight(row):
    if row['Train_Flag'] > 0:
        return pd.Series(['background-color: yellow']*len(row), index=row.index)
    else:
        return pd.Series(['']*len(row), index=row.index)


if(select_files == 'y'):
    # Walk through all files and directories within the folder
    for root, directories, files in os.walk(folder_path):
        for filename in files:
            # Append the path to each file to the file_paths list
            file_paths_details.append(os.path.join(root, filename))
    df4 = pd.DataFrame(file_paths_details)
    df4['Train_Flag'] = 0
    df4.columns = ["filepath","Train_Flag"]
    df4.to_csv("df4.csv", index=False)
    display(df4)
    print("Please Update the dataframe CSV File")
else:
    df4 = pd.read_csv('df4.csv')
    df4.columns = ["filepath","Train_Flag"]
    df5 = df4
    df5 = df5.style.apply(highlight, axis=1)
    display(df5)

Update File List? : n


,filepath,Train_Flag
0,repo/index.html,1
1,repo/.DS_Store,0
2,repo/login-page.js,1
3,repo/style.css,1


In [47]:
#read OPEN AI API KEY

text_file = open("API_key.txt", "r")
openai.api_key =  text_file.read()

text_file.close()

line_embeddings = []
    
def split_file_by_line(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        line_number = 0
        for i in lines:
            line_embeddings.append([filename,i,line_number])
            line_number +=1
    return 

comment_chars = {
    '.html':"<!-",
    '.css':"/*",
    '.js':"//",
    '.py':"#"
}

blocks = []
def split_file_by_character(filename,character):
    with open(filename, 'r') as f:
        lines = f.readlines()

    start_line = 0
    prev_end_line = -2
    for i, line in enumerate(lines):
        if character in line and prev_end_line != i-1 :
            stripped_line = line.replace(" ", "").replace("\t", "")
            #print(stripped_line[0:len(character)])
            if(character == stripped_line[0:len(character)]):
                prev_end_line = i
                extracted_lines = lines[start_line:prev_end_line]

                # join the extracted lines into a string
                extracted_text = "".join(extracted_lines)
                if extracted_text != "":
                    blocks.append([filename,start_line,prev_end_line,extracted_text])
                #print(filename)
                start_line = i 

    extracted_lines = lines[prev_end_line+1:len(lines)]
    # join the extracted lines into a string
    extracted_text = "".join(extracted_lines)
    blocks.append([filename,prev_end_line,len(lines),extracted_text])

    return 

for i in range(0,len(df4)):
    if (df4.iloc[i][1]==1):
        print(df4.iloc[i][0])
        split_file_by_line(df4.iloc[i][0])
        file_extension = os.path.splitext(df4.iloc[i][0])[1]
        split_file_by_character(df4.iloc[i][0],comment_chars[file_extension])

print("Total number of functions extracted:", len(blocks))

repo/index.html
repo/login-page.js
repo/style.css
Total number of functions extracted: 10


In [48]:
df = pd.DataFrame(line_embeddings)
df2 = pd.DataFrame(blocks)
df.columns = ["filepath","Code","LineNumber"]
display(df)
df2.columns = ["filepath","BlockStart","BlockStop","Code"]
display(df2)

,filepath,Code,LineNumber
0,repo/index.html,<!DOCTYPE html>\n,0
1,repo/index.html,"<html lang=""en"">\n",1
2,repo/index.html,\n,2
3,repo/index.html,<!- File References ->\n,3
4,repo/index.html,\n,4
...,...,...,...
130,repo/style.css,font-weight: bold;\n,81
131,repo/style.css,background-color: #33c1ea;\n,82
132,repo/style.css,cursor: pointer;\n,83
133,repo/style.css,outline: none;\n,84


,filepath,BlockStart,BlockStop,Code
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n"
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde..."
3,repo/index.html,22,32,"<form id=""login-form"">\n <input type=""tex..."
4,repo/login-page.js,0,17,"const loginForm = document.getElementById(""log..."
5,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n
6,repo/style.css,4,16,/* Body Styles */\n/* Main Styles */\nbody {...
7,repo/style.css,16,28,/* Holder for login items*/\n#main-holder {\n ...
8,repo/style.css,28,54,/* Login Error Message Styles */\n#login-erro...
9,repo/style.css,54,86,#login-form {\n align-self: flex-start;\n di...


In [ ]:
df['code_embedding'] = ''
def get_embedding(task):
    time.sleep(3)
    response = openai.Embedding.create(
            input=task,
            model="text-embedding-ada-002"
        )
    return response['data'][0]['embedding']
i=0
for ind in df.index:
        i+=1
        df['code_embedding'][ind] = get_embedding(df['Code'][ind])
        print(round(100*i/len(df)))
print("Done")
df.to_csv("df.csv", index=False)


/var/folders/tb/txlhvbmn2fgct223r9bxzyc80000gn/T/ipykernel_4092/937155510.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['code_embedding'][ind] = get_embedding(df['Code'][ind])


1
1
2
3
4
4
5
6
7
7
8
9
10
10
11
12
13
13
14
15
16
16
17
18
19
19
20
21
21
22
23
24
24
25
26
27
27
28
29
30
30
31
32
33
33
34
35
36
36
37
38
39
39
40
41
41
42
43
44
44
45
46
47
47
48
49
50
50
51
52
53
53
54
55
56
56
57
58
59
59
60
61
61
62
63
64
64
65
66
67
67
68
69
70
70
71
72
73
73
74
75
76
76
77
78
79
79
80
81


In [157]:
df = pd.read_csv('df.csv')
df['code_embedding'] = df.code_embedding.apply(lambda x: [float(y) for y in x[1:-1].split(",")])
display(df)

,filepath,Code,LineNumber,code_embedding
0,repo/index.html,<!DOCTYPE html>\n,0,"[-0.01256885752081871, 0.0028874874114990234, ..."
1,repo/index.html,"<html lang=""en"">\n",1,"[-0.009005490690469742, 0.010273775085806847, ..."
2,repo/index.html,\n,2,"[0.0025078936014324427, -0.018215738236904144,..."
3,repo/index.html,<!- File References ->\n,3,"[-0.025017328560352325, -0.0020156055688858032..."
4,repo/index.html,\n,4,"[0.0025078936014324427, -0.018215738236904144,..."
...,...,...,...,...
129,repo/style.css,font-weight: bold;\n,80,"[-0.020041272044181824, 0.0022615562193095684,..."
130,repo/style.css,background-color: #33c1ea;\n,81,"[-0.008634612895548344, -0.0038089891895651817..."
131,repo/style.css,cursor: pointer;\n,82,"[-0.034579064697027206, -0.0009556518634781241..."
132,repo/style.css,outline: none;\n,83,"[-0.03516428545117378, 0.01233269739896059, 0...."


In [161]:
import numpy 
import matplotlib
task = 'When submit button is clicked redirect javascript call OTP Auth page '
from openai.embeddings_utils import cosine_similarity

def search_functions(df, code_query):
    embedding = get_embedding(code_query)
    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))
    res = df.sort_values('similarities', ascending=False).head(100)
    return res

res = search_functions(df, task)
print("Done!")

Done!


In [162]:
res = df.sort_values('similarities', ascending=False).head(round(0.05*len(df)))
display(res)
print(len(res))

,filepath,Code,LineNumber,code_embedding,similarities
26,repo/index.html,"<input type=""submit"" value=""Login"" id=""log...",26,"[-0.020360980182886124, -0.0023479533847421408...",0.823025
34,repo/login-page.js,"const loginButton = document.getElementById(""l...",2,"[-0.02229473926126957, -0.009531905874609947, ...",0.815780
123,repo/style.css,#login-form-submit {\n,74,"[-0.014604851603507996, 0.010971074923872948, ...",0.804221
23,repo/index.html,"<form id=""login-form"">\n",23,"[-0.018279781565070152, 0.011316388845443726, ...",0.795506
37,repo/login-page.js,"loginButton.addEventListener(""click"", (e) => {\n",5,"[-0.02293883077800274, -0.016787413507699966, ...",0.791309
38,repo/login-page.js,e.preventDefault();\n,6,"[-0.017884865403175354, -0.02801126427948475, ...",0.790936
44,repo/login-page.js,location.reload();\n,12,"[-0.018045900389552116, -0.0164205189794302, -...",0.790369


7


In [97]:
df3 = pd.DataFrame()

In [96]:
df2["Hits"] = 0
display(df2)

,filepath,BlockStart,BlockStop,Code,Hits
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,0
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",0
3,repo/index.html,22,32,"<form id=""login-form"">\n <input type=""tex...",0
4,repo/login-page.js,0,17,"const loginForm = document.getElementById(""log...",0
5,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
6,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
7,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,0
8,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...,0
9,repo/style.css,53,85,#login-form {\n align-self: flex-start;\n di...,0


In [98]:
# group by filepath and concatenate LineNumber
df3 = res.groupby("filepath").agg({"LineNumber": list}).reset_index()

display(df3)

,filepath,LineNumber
0,repo/index.html,"[17, 8, 14, 23]"
1,repo/style.css,"[54, 53, 74]"


In [99]:
import numpy as np
def count_lines(filepath, start, stop):
    count = 0
    for i in df3[df3['filepath']==filepath]['LineNumber']:
        #print(i)
        for j in i:
            if(j>=start and j<stop):
                count +=1
    return count

# apply the function to each row of df2 and create a new column
df2['Hits'] = df2.apply(lambda row: count_lines(row['filepath'], row['BlockStart'], row['BlockStop']), axis=1)
display(df2)
df2.to_csv("df2.csv", index=False)


,filepath,BlockStart,BlockStop,Code,Hits
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,1
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",2
3,repo/index.html,22,32,"<form id=""login-form"">\n <input type=""tex...",1
4,repo/login-page.js,0,17,"const loginForm = document.getElementById(""log...",0
5,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
6,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
7,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,0
8,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...,0
9,repo/style.css,53,85,#login-form {\n align-self: flex-start;\n di...,3


In [100]:
display(df2)
df2.to_csv("df2.csv", index=False)

,filepath,BlockStart,BlockStop,Code,Hits
0,repo/index.html,0,3,"<!DOCTYPE html>\n<html lang=""en"">\n\n",0
1,repo/index.html,3,14,<!- File References ->\n\n<head>\n <meta char...,1
2,repo/index.html,14,22,"<!- Login Page Holder ->\n<main id=""main-holde...",2
3,repo/index.html,22,32,"<form id=""login-form"">\n <input type=""tex...",1
4,repo/login-page.js,0,17,"const loginForm = document.getElementById(""log...",0
5,repo/style.css,0,4,/* Page Styles */\nhtml {\n height: 100%;\n}\n,0
6,repo/style.css,4,15,/* Body Styles */\nbody {\n height: 100%;\n ...,0
7,repo/style.css,15,27,/* Holder for login items*/\n#main-holder {\n ...,0
8,repo/style.css,27,53,/* Login Error Message Styles */\n#login-erro...,0
9,repo/style.css,53,85,#login-form {\n align-self: flex-start;\n di...,3


In [101]:
df2 = df2.sort_values('Hits', ascending=False)   
print(df2.iloc[0])

filepath                                         repo/style.css
BlockStart                                                   53
BlockStop                                                    85
Code          #login-form {\n  align-self: flex-start;\n  di...
Hits                                                          3
Name: 9, dtype: object
